In [ ]:
#Simple experimentation trial of RAG 
# What im trying to do is to use the Chatbot OpenAI API to create a RAG system that can answer questions based on a set of documents.


In [2]:
import sys
print(sys.executable)


/Users/aditishelke/Downloads/github project 1/venv/bin/python


In [3]:
## Data Ingestion 
# Read form files pdf, web or whatever 
from langchain_community.document_loaders import TextLoader

loader = TextLoader("speech.txt")
text_documents = loader.load() #textdocument 
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='Power Pivot and the Data Model\n\nPower Pivot operates within a Data Model, where relationships are established and managed. This approach permits the creation of a PivotTable from multiple sources without physically merging them.\n\nMaking use of the DAX formula language, Power Pivot handles intricate calculations on the data model, spanning time intelligence, rankings, percentiles, and more.\n\nA major advantage lies in Power Pivot’s ability to efficiently manage numerous data sources. It doesn’t necessitate storing a memory-intensive “Frankentable” and computes DAX measures as needed. However, mastering Power Pivot can prove challenging due to its steep learning curve, particularly when dealing with data sources not consolidated into a single table.\n\nOn the sales_pp worksheet of ch_06.xlsx, I have created a Data Model consisting of the three sales data sources and loaded the results to a PivotTable. Now I am able to analyz

In [4]:

from dotenv import load_dotenv
import os

load_dotenv()  # Loads from .env automatically
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


In [5]:
# Data ingestion 
# Read from web based 
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )),)

text_documents=loader.load()



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
text_documents

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2024-04-12-diffusion-video/'}, page_content='\n\n      Diffusion Models for Video Generation\n    \nDate: April 12, 2024  |  Estimated Reading Time: 20 min  |  Author: Lilian Weng\n\n\nDiffusion models have demonstrated strong results on image synthesis in past years. Now the research community has started working on a harder task—using it for video generation. The task itself is a superset of the image case, since an image is a video of 1 frame, and it is much more challenging because:\n\nIt has extra requirements on temporal consistency across frames in time, which naturally demands more world knowledge to be encoded into the model.\nIn comparison to text or images, it is more difficult to collect large amounts of high-quality, high-dimensional video data, let along text-video pairs.\n\n\n\n🥑 Required Pre-read: Please make sure you have read the previous blog on “What are Diffusion Models?” for image generation before 

In [12]:
#Read from PDf directly 
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('/Users/aditishelke/Downloads/github project 1/temp_resume.pdf')


In [13]:
resume = loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)


In [14]:
resume

[Document(metadata={'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20250616201610Z00'00'", 'title': 'Aditi Shelke resume', 'author': 'Aditi Shelke', 'moddate': "D:20250616201610Z00'00'", 'source': '/Users/aditishelke/Downloads/github project 1/temp_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="Aditi Shelke aditishelke333@gmail.com |\tHoboken,\tNJ\t|\tlinkedin.com/in/aditi-shelke | github.com/AditiShelke  \nEDUCATION  \nStevens Institute of Technology                                                                                                                                                                        Expected May 2026 Master of Science in Data Science Relevant Coursework: Probability, Machine Learning, Data Mining, Data Visualization, Marketing Analytics  D.Y.Patil College of Engineering & Technology, Kolhapur           Sept 2024 Bachelor of Technology | CGPA: 8.49 Relevant Coursework: 

In [15]:
#Transform as per the pipeline 
#can do for other files too excel, etc etc check RAG pipeline 

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(resume)
documents[:5]

[Document(metadata={'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20250616201610Z00'00'", 'title': 'Aditi Shelke resume', 'author': 'Aditi Shelke', 'moddate': "D:20250616201610Z00'00'", 'source': '/Users/aditishelke/Downloads/github project 1/temp_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Aditi Shelke aditishelke333@gmail.com |\tHoboken,\tNJ\t|\tlinkedin.com/in/aditi-shelke | github.com/AditiShelke  \nEDUCATION'),
 Document(metadata={'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20250616201610Z00'00'", 'title': 'Aditi Shelke resume', 'author': 'Aditi Shelke', 'moddate': "D:20250616201610Z00'00'", 'source': '/Users/aditishelke/Downloads/github project 1/temp_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Stevens Institute of Technology                                                                             

In [16]:
documents

[Document(metadata={'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20250616201610Z00'00'", 'title': 'Aditi Shelke resume', 'author': 'Aditi Shelke', 'moddate': "D:20250616201610Z00'00'", 'source': '/Users/aditishelke/Downloads/github project 1/temp_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Aditi Shelke aditishelke333@gmail.com |\tHoboken,\tNJ\t|\tlinkedin.com/in/aditi-shelke | github.com/AditiShelke  \nEDUCATION'),
 Document(metadata={'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20250616201610Z00'00'", 'title': 'Aditi Shelke resume', 'author': 'Aditi Shelke', 'moddate': "D:20250616201610Z00'00'", 'source': '/Users/aditishelke/Downloads/github project 1/temp_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Stevens Institute of Technology                                                                             

In [17]:
# Vector embeddings wrt OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings())


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [18]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192"  
)


In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(documents, embedding_model)

/var/folders/jm/560w9m8d6xngldzyq4s7_v6w0000gn/T/ipykernel_2235/3816544679.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/aditishelke/Downloads/github project 1/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import os

# Load .env for GROQ_API_KEY
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")


# Step 2: Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(resume)

# Step 3: Embed using HuggingFace
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Store in Chroma
db = Chroma.from_documents(chunks, embedding_model, persist_directory="chroma_resume_db")
retriever = db.as_retriever()

# Step 5: LLM via Groq
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192"
)

# Step 6: Build RAG pipeline
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Step 7: Ask question
while True:
    query = input("Ask me about my resume (or type 'exit'): ")
    if query.lower() == "exit":
        break
    answer = qa.run(query)
    print(f"\n🤖 Answer: {answer}\n")


/var/folders/jm/560w9m8d6xngldzyq4s7_v6w0000gn/T/ipykernel_2235/1473561502.py:40: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(query)



🤖 Answer: Here's a sample cover letter based on the provided context:

Dear [Hiring Manager's Name],

I am excited to apply for the [Position] role at [Company Name]. As a detail-oriented and analytical individual with a strong passion for data science, I am confident that my skills and experience make me an ideal fit for your team.

Currently pursuing my Master's in Data Science at Stevens Institute of Technology, I have developed a solid foundation in probability, machine learning, data mining, and data visualization. My undergraduate degree in Bachelor of Technology from D.Y. Patil College of Engineering & Technology has provided me with a strong understanding of data principles. Additionally, I have augmented my skills with certifications in AWS Academy Cloud Foundations and Cloud Architecting, as well as Coursera's Python for Data Science and AI.

In my current role as a Data Analyst at The Compass Group USA, I have successfully developed forecasting dashboards using Power BI and

In [ ]:
#see hpow can you improve this pipeline and make it more efficient and faster
# --- IGNORE ---
# This is a simple RAG pipeline that can be extended with more sophisticated techniques like caching,
# more advanced vector stores, or even integrating with other data sources.
# --- IGNORE ---
# Resume madhe aan using ollama embeddingss and vector store 



SyntaxError: invalid syntax (1109794240.py, line 8)

In [ ]:
#agents 
# sequence of actions to take a sequence of action 
# langchain already has soem generic prompts created by people 
